In [1]:
from hedypet.utils import DERIVATIVES_ROOT, load_splits, get_time_frames_midpoint
from nifti_dynamic.utils import load_tac
from nifti_dynamic.patlak import roi_patlak
from parse import parse
from tqdm import tqdm
import warnings
import pandas as pd 
import os 

In [2]:
subs = load_splits()["all"]

In [3]:
if not os.path.exists(df_path := "acstatPSF_means.pkl"):
    data = []

    for sub in tqdm(subs):
        tacs_root = (DERIVATIVES_ROOT / f"tacs/{sub}/acstatPSF")
        tacs = list(tacs_root.glob("**/tac*"))

        #IF tacs not relevant for mean analysis
        tacs_if = [x for x in tacs if "aorta" in str(x)]
        tacs_organs = [x for x in tacs if x not in tacs_if]
                
        # Try all combinations of inputs functions and num_frames
        for tac_organ_path in tacs_organs:
            mu_organ, std_organ, n_organ = load_tac(tac_organ_path)
            tags = parse('{}/tacs/{sub}/acstatPSF/{task}/erosion-{erosion}/tac_{ix}',str(tac_organ_path)).named
            vals = {"mu":float(mu_organ),"std":float(std_organ),"n":int(n_organ)}
            vals.update(tags)
            data.append(vals)

    df = pd.DataFrame(data)
    df.to_pickle(df_path)
else:
    df = pd.read_pickle(df_path)

In [8]:
from hedypet.utils import get_norm_consts

print("columns:", df.columns)
print("Unique task:", df.task.unique())
print("Unique erosion:", df.erosion.unique())
print("Available normalizations:", list(get_norm_consts("sub-000").keys()))
print("Rows:",len(df))
print("Subjects:", df["sub"].nunique())

columns: Index(['mu', 'std', 'n', 'sub', 'task', 'erosion', 'ix'], dtype='object')
Unique task: ['ts_total' 'synthseg' 'synthsegparc' 'ts_tissue' 'ts_body' 'totalimage']
Unique erosion: ['0' '1']
Available normalizations: ['suv', 'sul_janma', 'sul_james', 'sul_decazes', 'sul_auto', 'luv', 'sul_auto_decazes']
Rows: 49750
Subjects: 100


,mu,std,n,sub,task,erosion,ix
0,665.961304,176.173401,51140,sub-000,ts_total,0,1
1,1276.164185,863.834595,35163,sub-000,ts_total,0,2
2,1190.210327,628.152954,36571,sub-000,ts_total,0,3
3,469.122833,163.980255,3583,sub-000,ts_total,0,4
4,929.510559,254.466522,341770,sub-000,ts_total,0,5
...,...,...,...,...,...,...,...
49248,532.304199,1017.503052,6857523,sub-099,ts_body,0,1
49249,627.355774,869.750366,2313968,sub-099,ts_body,0,2
49250,540.169739,1033.212524,6634912,sub-099,ts_body,1,1
49251,655.252075,899.339233,2126582,sub-099,ts_body,1,2
